# 最尤法と最小二乗法

出力$y_i$と入力$x_i$の関係($i=1,2,\cdots,N$)を、線形式$a_0+ax_i$と正規分布に従う誤差$\epsilon_i \sim N(0,\sigma^2)$により、モデル化(**単回帰モデル**)することを考えます。
$$y_i=a_0+ax_i+\epsilon_i$$
$$\epsilon_i \sim N(0,\sigma^2)$$

このモデルとパラメータ$(a,a_0)$の下で、N個のペアデータ$(x_i,y_i)$が得られる確率は、
$$L(a_0,a)=\prod_{i=1}^{N} \frac{1}{\sqrt{2\pi\sigma^2}} \exp(-\frac{(y_i-a_0-ax_i)^2}{2\sigma^2})$$
となります。
モデルパラメータ$a_0,a$を変えることで、$L(a_0,a)$を最大にするようなパラメータを求め。それをモデルパラメータとする推定方法を**最尤推定**とよびます。
$$(\hat{a_0},\hat{a})=\arg \max_{a_0,a} L(a_0,a)$$

対数は単調増加関数であるので、
$$(\hat{a_0},\hat{a})=\arg \max_{a_0,a} \log L(a_0,a)=\arg \min_{a_0,a}\{ -\log L(a_0,a)\}$$
であり、負の対数尤度を最小化する問題と同じとなります。
$$-\log L(a_0,a)=\frac{N}{2}\log{(2\pi\sigma^2)}+\frac{1}{2\sigma^2}\sum_{i=1}^{N} (y_i-a_0-ax_i)^2$$
であることから、
$$(\hat{a_0},\hat{a})=\arg \min_{a_0,a}\{ -\log L(a_0,a)\}=\arg \min_{a_0,a}\sum_{i=1}^{N} (y_i-a_0-ax_i)^2$$

よって、誤差を正規分布と仮定する線形モデルにおいて、最尤法によりパラメータを求める方法は、最小二乗法によるものと一致します。

ここまでは、説明変数が1つのモデルを考えましたが、説明変数が$k$個あるようなモデルにも拡張することができ、**重回帰モデル**とよびます。
$$y_i \sim N(a_0+a_1 x_{1,i}+a_2 x_{2,i}+\cdots,a_k x_{k,i},\sigma^2)$$
$$(\hat{a_0},\hat{a_1},\cdots,\hat{a_k})=\arg \min_{a_0,a_1,\cdots,a_k}\sum_{i=1}^{N} (y_i-a_0-a_1 x_{1,i}-\cdots-a_k x_{k,i})^2$$

さて、最適なパラメータは、尤度関数の各パラメータについての偏微分が0となるようなパラメータとなります。
ここで、$$\vec{a}=\{a_0,a_1,\cdots,a_k\}^T, \vec{y}=\{y_1,y_2,\cdots,y_N\}^T, 
\tilde{X}= \left(
    \begin{array}{ccccc}
      1 & x_{1,1} & x_{1,2} & \cdots & x_{1,k} \\
      1 & x_{2,1} & x_{2,2} & \cdots & x_{2,k} \\
      \vdots & \vdots & \vdots & \ddots & \vdots \\
      1 & x_{N,1} & x_{N,2} & \cdots & x_{N,k}
    \end{array}
  \right)$$
とおくと、以下の関係(正規方程式)が成り立ちます。
$$\tilde{X} y = \tilde{X}^T \tilde{X} \hat{a}$$
したがって、$\hat{a}$は以下の様に求められます。
$$\vec{\hat{a}}=(\tilde{X}^T\tilde{X})^{-1} \tilde{X}^T \vec{y}$$


## 演習
単回帰モデルによる分析を試してみるため、$y=0.1x+1$となるような人工データの生成してみます。

In [ ]:
import numpy as np
import statsmodels.api as sm

N = 100                     # 生成するデータ数
a = np.array([1, 0.1])      # 切片1, 傾き0.1
x = np.linspace(0, 10, N)   # xとして0~10までN個のデータを生成
e = np.random.normal(scale=.2,size=N)    # 正規分布に従う誤差を生成
X = sm.add_constant(x)      # 切片a_0を推定するため、Xに1の列を追加
y_true=np.dot(X, a)         # y_true=aXを計算
y = y_true + e              # 観測されるyは、y_trueにノイズeが乗ったもの

生成した人工データから、パラメータを推定してみます。

In [ ]:
model = sm.OLS(y, X)    # y=aXというモデルを作成
result = model.fit()    # パラメータaを推定
print(result.summary()) # 推定したパラメータを表示

推定された結果を見てみましょう。coefが偏回帰係数の最尤推定量を意味し、constが切片,x1が傾きです。 それぞれ1、0.1に近い値が推定されているはずです。また、[0,025 0.975]が各パラメータの95%信頼区間です。95%の確率で、1や0.1がその範囲に含まれるような結果が出ているはずです。

R-squaredが決定係数と呼ばれるもので、0以上1以下の値を取り、1に近いほど当てはまりが良いことを示します。今回のような単回帰モデルでは相関係数の2乗と一致します。

真の関係(y=0.1x+1)を青線で、観測値を青点で、推定した関係を赤の破線で描くと以下のグラフが得られます。

In [ ]:
import matplotlib.pyplot as plt
plt.plot(x, y, 'o', label="Observed")
plt.plot(x, y_true, 'b-', label="True")
plt.plot(x, result.fittedvalues, 'r-.', label="Estimated")
plt.legend(loc='best')
plt.show()

---
#多重共線性と正則化

説明変数$x_i=\{x_{1,i},x_{2,i},\cdots x_{N,i}\}$と$x_j=\{x_{1,j},x_{2,j},\cdots x_{N,j}\}$の相関が高い場合を考えます。
このとき、$det(X^TX) \simeq 0$
となり、$X^TX$の逆行列を求める際に0に近い除算が生じることから、$a$の推定値が不安定となる等の不具合が生じます。
これを、多重共線性の問題とよびます。

多重共線性の問題の解決策の一つとしては、相関が高い変数が複数ある場合は、どれか1つだけ残す方法があります。

別の方法として、正則化とよばれる手法があります。
ここで、以下の最小化問題を考えます。
$$\min_{a_0,a_1,\cdots,a_k} \{\sum_{i=1}^{N} (y_i-a_0-a_1 x_{1,i}-\cdots-a_k x_{k,i})^2+\lambda \sum_{i=0}^{k} a_k^2\}$$
第1項は、これまでの最小二乗法と同じ式であり、予測値の当てはまりの良いモデルほど小さな値を取ります。
第2項は、正則化項とよばれるものであり、偏回帰係数が0に近いほど、小さな値を取ります。
よって、予測値の当てはまりの良さだけでなく、偏回帰係数の0への近さとの両立が取れたモデルが選択されることになります。
この最適解は、
$$a=(X^TX+\lambda I)^{-1} X^T y$$
となることから、適切な$\lambda$を設定することにより、0除算に起因する多重共線性の問題は解決されることになります。
今回の正則化では、正則化項に偏回帰係数の2乗(L2ノルム)を用いましたが[L2正則化]、偏回帰係数の絶対値(L1ノルム)を使う方法[L1正則化]もあります。
$$\min_{a_0,a_1,\cdots,a_k} \{\sum_{i=1}^{N} (y_i-a_0-a_1 x_{1,i}-\cdots-a_k x_{k,i})^2+\lambda \sum_{i=0}^{k} |a_k|\}$$

L1正則化を用いた線形回帰モデルは**LASSO回帰**、L2正則化を用いた線形回帰モデルは**Ridge回帰**とも呼ばれ、機械学習の分野などで広く使われます。

##演習

PythonにおけるLASSOおよびRidge回帰は、線形回帰の関数を変えるだけで実行できます。
線形回帰では、fit()関数を用いましたが、LASSO/Ridgeでは、以下のようにfit_regularized()関数を用いることで実現できます。

In [ ]:
result.lasso = model.fit_regularized(
                    method='elastic_net',   # LidgeとLASSOを混合したモデルがElastic Net
                    alpha=0.5,    # 正則化パラメータλ
                    L1_wt=1.0,    # 1にするとLASSO (0.5にするとLASSOとRidgeが半々)
                    start_params=None,
                    profile_scale=False,
                    refit=False
                    )
result.ridge = model.fit_regularized(
                    method='elastic_net',
                    alpha=0.5,    # 正則化パラメータλ
                    L1_wt=0.0,    # 0にするとRidge
                    start_params=None,
                    profile_scale=False,
                    refit=False
                    )

print("OLS {}".format(result.params))
print("LASSO: {}".format(result.lasso.params))
print("ridge: {}".format(result.ridge.params) )

OLSでは推定された切片が1.0前後ですが、正則化による影響でRidgeではさらに小さく、LASSOでは0になっていることが分かります。LASSOは、偏回帰係数を0ちょうどにしやすい性質があることから、変数選択法としても用いられます。

---
#ロジスティック回帰
今、ある現象が起きるか起きないかをモデル化することを考えます。
現象の起きやすさを表す指標$t$を考え、
以下のような線形式で表されるとします。
$$t=a_0+a_1 x_1+\cdots a_k x_k$$
$t$がロジスティック関数の一種であるシグモイド関数$\sigma()$により、確率$p$に変換されるものとします。
$$p=\sigma(t)=\frac{\exp(t)}{1+\exp(t)}=\frac{1}{1+\exp(-t)}$$
$t=+\infty$で$p=1$、$t=0$で$p=0.5$、$t=-\infty$で$p=0$となります。

この$p$によって、現象が起きるかどうか決まるので、
$$P(Y=y|X=x)=P(Y=1|X=x)^yP(Y=0|X=x)^{1-y}=\sigma(t)^y(1-\sigma(t))^{1-y}$$
以上のように、ある現象が起きるか起きないかをモデル化したものを**ロジスティック回帰モデル**とよびます。

尤度関数は、
$$L(\vec{a})=\prod_{i=1}^N [\sigma(\vec{a}x_i)^{y_i} (1-\sigma(\vec{a}x_i))^{1-y_i}]$$
となり、これを最大とするようなパラメータを求めることになります。



## ロジスティック回帰モデルにおける偏回帰係数の意味
ある現象が起きる確率$p$と起きない確率$1-p$の比$p/(1-p)$をオッズと呼びます。
$p$が大きいほどオッズは大きくなり、$p$が小さいほどオッズは小さくなる関係にあります。
ロジスティック回帰モデルにおいては、
$$\frac{p}{1-p}=\exp(t)$$
です。説明変数$x$を1増やした後$p_1$とその前$p_2$のオッズの比を求めると、
$$ \frac{\frac{p_1}{1-p_1}}{\frac{p_2}{1-p_2}}=\frac{\exp\{a(x+1)+a_0)\}}{\exp(ax+a_0)}=\exp(a)$$
となります。
$a>0$の場合、$\exp(a)>1$となりますから、$p_1>p_2$となります。
すなわち、$x$の値が大きいほどその現象は起きやすくなります。
逆に、$a<0$の場合は$p_1<p_2$ですから、$x$の値が大きくなるにつれ、その現象は起きにくくなります。
すなわち、回帰係数$a$の符号により、その説明変数がアウトカムに与える影響を調べることができます。


---
# 総合演習課題

プロ野球選手の年俸がどのように決まっているのか調べたいと考えた。
そこで以下の仮説を立てた。
- ある年の年俸は、その前年の成績によって決まる
- ホームラン1本、ヒット1本、盗塁1回などについて、それぞれ単価が決まっており、それらを合計したものが年俸となる。すなわち、ホームランやヒット数、盗塁数を説明変数、目的変数を推定年俸とする重回帰モデルを仮定する

重回帰モデルを構築するため、選手の年俸データおよび成績データを[プロ野球データFreak](https://baseball-data.com/)から取得した。このデータを加工して、解析しやすくしたものが、
* 選手概要: player2016.csv ～ player2020.csv
* 打撃成績: batter2016.csv ～ batter2020.csv
* 投手成績: pitcher2016.csv ～ pitcher2020.csv

の計15個あり、今回はplayer2020とbatter2019/pitcher2019を読み込みます。


In [ ]:
import pandas as pd
player2020 = pd.read_csv('http://ads.cs.kumamoto-u.ac.jp/csv/player2020.csv')     #2020年開幕時の選手概要
batter2019 = pd.read_csv('http://ads.cs.kumamoto-u.ac.jp/csv/batter2019.csv')     #2019年シーズンの打撃成績
pitcher2019 = pd.read_csv('http://ads.cs.kumamoto-u.ac.jp/csv/pitcher2019.csv')   #2019年シーズンの投手成績

print(player2020)
print(batter2019)
print(pitcher2019)

選手概要について集計してみましょう。

In [ ]:
player2020.describe()

2020年の全933選手の年俸平均は4331万円、最高額は65000万円(=6億5000万円)、中央値は　1400万円、最低額は230万円であった。

年俸情報はplayer2020に、成績はbatter2019/pitcher2019に分かれて格納されてある。
そこで、まず年俸情報と打者成績を(球団名,選手名)に基づいて結合した。



In [ ]:
seiseki19 = pd.merge(player2020,batter2019)   # player2020とbatter2019で結合。特に指定しないと、共通する項目名(球団,選手名)が結合キーとして自動的に選ばれる
print(seiseki19)

結合したデータを用いて、ホームラン数、ヒット数を説明変数、推定年俸を目的変数とする重回帰モデルを構築する。

In [ ]:
x=seiseki19[["本塁打","安打","盗塁"]]
X = sm.add_constant(x)
model = sm.OLS(seiseki19["推定年俸"], X)    # y=aXというモデルを作成
result = model.fit()    # パラメータaを推定
print(result.summary()) # 推定したパラメータを表示

決定係数(R-squared)が0.533あることから、簡易的なモデルとはいえ、予測精度としては一定程度確保できている。

解析結果によると、本塁打の偏回帰係数が541、安打が55、盗塁が39と推定されていた。すなわち、本塁打が1本増えると541万円、安打1本増えると55万円年俸が増えると考えられる。本モデルにおいては、ホームラン1本はヒット約10本分の価値があると評価されている。

次に、偏回帰係数の95%を見てみる。本塁打・安打の95%信頼区間には0が含まれていない。よって、有意水準5%で「本塁打(もしくは安打)は年俸に寄与していない(=偏回帰係数が0)」とする帰無仮説は棄却され、「本塁打(もしくは安打)は年俸に寄与している」との対立仮説が採択される。一方で、盗塁の95%信頼区間には0が含まれており、「盗塁は年俸に寄与していない」とする帰無仮説は棄却されないことになる。

## 課題1
2020年データを用いて、日本出身者と外国出身者で推定年俸の平均に差があるか、すなわち
1. 日本出身者と外国出身者という2つの区分で、平均年俸の差があるか(その理由が成績差等に起因するものであっても良い)
2. どちらか一方の出身者の方が、成績が良いという理由のみでなく、成績差だけでは説明できない差が存在するか
の2点について議論しなさい。なお、都道府県名一覧のcsvデータ
```
todofuken = pd.read_csv('http://ads.cs.kumamoto-u.ac.jp/csv/todofuken.csv')
```
を使ってもよい。

注意: 今回のデータには全プロ野球選手のデータが含まれているため、標本データではなく母集団のデータとみることも可能である。ただ、推定年俸はその名の通り推定値であり、推定者やそのタイミング等によって値がばらつくものである。よって、標本データとして扱うこと。

## 解答1
(課題1に対する解答を以下に記述しなさい。必要であれば、テキストやコートを追加しても構いません)

## 課題2
プロ野球に関する何らかの仮説を自分で立ててみて、その仮説の正しさを今回のデータを用いて検証しなさい。


[検証例] プロ野球選手に早生まれ(1月～3月)は少ないが、平均年俸は他の月の生まれの選手と比較して差があるか

## 解答2
(課題2に対する解答を以下に記述しなさい。必要であれば、テキストやコートを追加しても構いません)

## 課題3
自分独自で視点を設定して、今回のデータの可視化を実施しなさい。

[実施例] 過去5年間の、球団ごとの総年俸の推移のグラフを作成する

## 解答3
(課題3に対する解答を以下に記述しなさい。必要であれば、テキストやコートを追加しても構いません)

## コンテスト課題

作成した打者の年収モデルの予測精度を向上させるため、モデルの改善を行うことを考える。次に挙げるものは、改善案の一例である。

* まず打率は説明変数に入れるべきだ。さらに打率が、2割から3割になった場合と3割から4割になった場合、どちらも1割の増加だが、評価としては後者の方が良いはずだ。打率を2乗したものも説明変数にすべきだ。また、首位打者になった等の打率の順位も重要でないか
* 年俸は、前年の成績だけでなく、前年以前の年俸も考慮して決まっているはずだ
* 強打者は、筋力が求められる一方で、走力はあまり求められず、比較的体重が重いはずだ。したがって、体重を強打者の代理変数として説明変数にいれることで精度の改善が図れるのではないか

このほかにも様々な改善策が考えられます。
今ある情報から作り出した新たな情報を説明変数として加える等、説明変数に様々な工夫を加えて、予測精度向上を図ることを**特徴量エンジニアリング**と呼びます。

それでは、以下の3つの課題を実施しなさい。

1. 特徴量エンジニアリングを施した打者の年俸推定モデルを構築しなさい。そして、どのような工夫を施したか、それによってどの程度予測精度が向上したか、報告しなさい。
2. 同様に、特徴量エンジニアリングを施した投手の年俸推定モデルを構築し、報告しなさい。
3. 作成した打者と投手の年俸予測モデル、および2020年の選手成績(hitter2020.csv/pitcher2020.csv)を用いて、各選手の2021年の年俸を予測しなさい。そして、予想年俸が高いTop100の選手情報をcsvにまとめmoodleから提出しなさい。ただし、提出するcsvの形式は[sampleRank.csv](http://ads.cs.kumamoto-u.ac.jp/csv/sampleRank.csv)の例に従うこと。

## 課題の評価について
1と2の、年俸推定モデル構築については、解析レポートの出来によって評価します。
3については、
1. 本当の年俸ランキングと比較することで、各学生が予測したランキングの予測の正しさを定量化
2. 予測が正しかった順(予測が同順の場合は、レポート提出が早い順)に学生を並び替え、上位の学生に加点する

とします。

補足: 2020年は新型コロナウィルスの影響により、プロ野球の開幕が遅れ、試合数も例年よりも少なくなっています(143->120)。選手年俸総額が減額される可能性もあり、年俸予測がいつも以上に難しいため、年俸ランキングという比較的影響を受けにくい指標を用います。